<a href="https://colab.research.google.com/github/sprerak48/MIT-Fintech-Challenge/blob/master/Team_TeamSeven_wrapper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score

**read_data_small** is the function to read in the small dataset about 30 MB

In [0]:
def read_data_small():
    X_train = pd.read_csv("X_train_small.csv")
    X_test = pd.read_csv("X_test_small.csv")
    y_train = np.asarray(pd.read_csv("y_train_small.csv", header=None)[0])
    return X_train, X_test, y_train

**detect_spoofying** is the function for training the classifier and classify the results. 

Here we provide an simple example.

In [0]:
### import libraries here ###
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import scale
from sklearn.model_selection import cross_validate
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.svm import OneClassSVM
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
### code classifier here ###
def format_data(df):

    # append numberical columns
    rst = df.loc[:,["price","volume","bestBid","bestAsk",'bestBidVolume',
                    'bestAskVolume','lv2Bid', 'lv2BidVolume','lv2Ask', 
                    'lv2AskVolume', 'lv3Bid', 'lv3BidVolume', 'lv3Ask',
                    'lv3AskVolume']]
    
    # encode the binaries
    rst["isBid"] = df.isBid*1
    rst["isBuyer"] = df.isBuyer*1
    rst["isAggressor"] = df.isAggressor*1
    rst["type"] = (df.type == "ORDER")*1
    rst["source"] = (df.source=="USER")*1
    
    # parse the order id data
    rst["orderId"] = df.orderId.str.split('-').str[-1]
    rst["tradeId"] = df.tradeId.str.split('-').str[-1]
    rst["bidOrderId"] = df.bidOrderId.str.split('-').str[-1]
    rst["askOrderId"] = df.askOrderId.str.split('-').str[-1]
    
    # encode the multiple lable data
    tmp_operation = pd.DataFrame(pd.get_dummies(df.operation), columns=df.operation.unique()[:-1])
    rst = pd.concat([rst, tmp_operation], axis=1)
    tmp_endUserRef = pd.DataFrame(pd.get_dummies(df.endUserRef), columns=df.endUserRef.unique()[:-1])
    rst = pd.concat([rst, tmp_endUserRef], axis=1)
    
    return rst

def detect_spoofying(X_train, X_test, y_train):
    
    # clean up the data
    X_concat = pd.concat([X_train, X_test])
    X_clean = format_data(X_concat)
    
    # feel free to add more columns inferred from data
    # smartly engineered features can be very useful to improve the classification resutls
    X_clean["timeSinceLastTrade"] = pd.concat([X_train.loc[:,["timestamp","endUserRef"]].groupby("endUserRef").diff(),
                                              X_test.loc[:,["timestamp","endUserRef"]].groupby("endUserRef").diff()])
    
    X_clean = X_clean.fillna(-1)
    X_train_clean = X_clean.iloc[:X_train.shape[0],:]
    X_test_clean = X_clean.iloc[X_train.shape[0]:,:]
    X_train_clean_scaled = scale(X_train_clean)
    X_test_clean_scaled = scale(X_test_clean)

    #xgboost
    model = XGBClassifier().fit(X_train_clean_scaled, y_train,eval_metric="logloss")
    y_train_pred_x = model.predict_proba(X_train_clean_scaled)
    y_test_pred_x = model.predict_proba(X_test_clean_scaled)

    #naive bayes
    classifier = GaussianNB().fit(X_train_clean_scaled, y_train)
    y_train_pred_n = model.predict_proba(X_train_clean_scaled)
    y_test_pred_n = classifier.predict_proba(X_test_clean_scaled)

    #knn
    cl = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2).fit(X_train_clean_scaled, y_train)
    y_train_pred_k = model.predict_proba(X_train_clean_scaled)
    y_test_pred_k = cl.predict_proba(X_test_clean_scaled)

    #MLP
    clf = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(5, 2), random_state=1).fit(X_train_clean_scaled, y_train)
    y_train_mlp = clf.predict_proba(X_train_clean_scaled)
    y_test_mlp = clf.predict_proba(X_test_clean_scaled)

    return  y_train_pred_x, y_test_pred_x,  y_train_mlp, y_test_mlp 




In [0]:
### import libraries here ###
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import scale
from sklearn.model_selection import cross_validate
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.svm import OneClassSVM
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
### code classifier here ###
def format_data(df):

    # append numberical columns
    rst = df.loc[:,["price","volume","bestBid","bestAsk",'bestBidVolume',
                    'bestAskVolume','lv2Bid', 'lv2BidVolume','lv2Ask', 
                    'lv2AskVolume', 'lv3Bid', 'lv3BidVolume', 'lv3Ask',
                    'lv3AskVolume']]
    
    # encode the binaries
    rst["isBid"] = df.isBid*1
    rst["isBuyer"] = df.isBuyer*1
    rst["isAggressor"] = df.isAggressor*1
    rst["type"] = (df.type == "ORDER")*1
    rst["source"] = (df.source=="USER")*1
    
    # parse the order id data
    rst["orderId"] = df.orderId.str.split('-').str[-1]
    rst["tradeId"] = df.tradeId.str.split('-').str[-1]
    rst["bidOrderId"] = df.bidOrderId.str.split('-').str[-1]
    rst["askOrderId"] = df.askOrderId.str.split('-').str[-1]
    
    # encode the multiple lable data
    tmp_operation = pd.DataFrame(pd.get_dummies(df.operation), columns=df.operation.unique()[:-1])
    rst = pd.concat([rst, tmp_operation], axis=1)
    tmp_endUserRef = pd.DataFrame(pd.get_dummies(df.endUserRef), columns=df.endUserRef.unique()[:-1])
    rst = pd.concat([rst, tmp_endUserRef], axis=1)
    
    return rst

def detect_spoofying_2(X_train, X_test, y_train):
    
    # clean up the data
    X_concat = pd.concat([X_train, X_test])
    X_clean = format_data(X_concat)
    
    # feel free to add more columns inferred from data
    # smartly engineered features can be very useful to improve the classification resutls
    X_clean["timeSinceLastTrade"] = pd.concat([X_train.loc[:,["timestamp","endUserRef"]].groupby("endUserRef").diff(),
                                              X_test.loc[:,["timestamp","endUserRef"]].groupby("endUserRef").diff()])
    
    X_clean = X_clean.fillna(-1)
    X_train_clean = X_clean.iloc[:X_train.shape[0],:]
    X_test_clean = X_clean.iloc[X_train.shape[0]:,:]
    X_train_clean_scaled = scale(X_train_clean)
    X_test_clean_scaled = scale(X_test_clean)


    

In [0]:
def wrapper():
    # read in data
    X_train, X_test, y_train = read_data_small()
    # or if you have the computational power to work with the big data set, 
    # you can comment out the read_data_samll line and uncomment the following read_data_big
    # X_train, X_test, y_train = read_data_big()
    
    # process the data, train classifier and output probability matrix
    y_train_pred_x, y_test_pred_x,  y_train_mlp, y_test_mlp  = detect_spoofying(X_train, X_test, y_train)
    
    # return the prediciton values
    return y_train_pred_x, y_test_pred_x,  y_train_mlp, y_test_mlp 

Call function wrapper:

In [0]:
y_train_pred_x, y_test_pred_x,  y_train_mlp, y_test_mlp  = wrapper()

In [0]:
pd.DataFrame(y_test_mlp).to_csv('MLP prediction probabilities.csv')
pd.DataFrame(y_test_mlp).to_csv('XGBoost prediction probabilities.csv')